# Projet Optimisation - Partie 2
---

## Imports

In [2]:
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize) # To print more elements of an array in terminal
import pandas as pd
from scipy.optimize import linprog
import matplotlib.pyplot as plt

# For modelising more complex problems
import cvxpy as cp

## Parameters

## Datas

In [3]:
# Loads dataframe
sites_df = pd.read_csv('Data-partie-2/Sites.csv')
offshore_df = pd.read_csv('Data-partie-2/Rendements_offshore.csv', header=None).to_numpy()
onshore_df = pd.read_csv('Data-partie-2/Rendements_onshore.csv', header=None).to_numpy()

# Get index of onshore and offshore sites and convert to numpy array
offshore_idx = sites_df[sites_df['capacite offshore'] == 'Oui']['index site'].to_numpy(dtype=int)
onshore_idx = sites_df[sites_df['capacite offshore'] == 'Non']['index site'].to_numpy(dtype=int)

# Get number of sites (n) and number of hours in a year (m)
n = sites_df.shape[0]
m = offshore_df.shape[1]

# Get max capacity of offshore and onshore sites
max_capacity = np.zeros(n)
max_capacity[offshore_idx] = sites_df[sites_df['capacite offshore'] == 'Oui']['capacites'].to_numpy(dtype=float)
max_capacity[onshore_idx] = sites_df[sites_df['capacite offshore'] == 'Non']['capacites'].to_numpy(dtype=float)

# Filter offshore and onshore dataframes
offshore_df = offshore_df[offshore_idx, :]
onshore_df = onshore_df[onshore_idx, :]
rendements_on_off = np.zeros((n, m)) # Matrix that contains all the rendements of onshore and offshore sites at the correct indices
rendements_on_off[offshore_idx, :] = offshore_df
rendements_on_off[onshore_idx, :] = onshore_df

# Loads consumption and hydro production
consumptions = pd.read_csv('Data-partie-2/Consommations.csv', header=None).to_numpy()
hydro_productions = pd.read_csv('Data-partie-2/Apports-hydro.csv', header=None).to_numpy()

# Get number of country (p)
p = consumptions.shape[0]

# Costs
onshore_install_cost = 168903 # [euro/MWh/year]
offshore_install_cost = 300336 # [euro/MWh/year]

gaz_install_cost = 94956 # [euro/MW/year]
gaz_production_cost = 65 # [euro/MWh]

# Country max stock per country [MWh]
max_stock = np.array([0.3*1e6 , 3.2*1e6 , 0.01*1e6 , 0 , 18.4*1e6 , 9.8*1e6 , 0.24*1e6 , 7.9*1e6 , 0.005*1e6 , 84.147*1e6 , 0 , 2.6*1e6 , 1.2*1e6 , 33.756*1e6 , 8.4*1e6])

# Max turbine and pump power per country [MW] 
max_turbine_power = np.array([8587 , 12009 , 1417 , 9 , 18372 , 25132 , 527 , 21117 , 1140 , 28941 , 37 , 5052 , 4269 , 16637 , 15101])
max_pump_power = np.array([5223 , 3580 , 1307 , 0 , 5347 , 4303 , 292 , 7544 , 1100 , 1396 , 0 , 1029 , 2744 , 45 , 1636 ])

# Turbine efficiency (eta)
turbine_efficiency = 0.85

## Question 4

Nous posons :
$$
\begin{align*}
    c_i &= \text{capacités sur le i}^\text{ème} \text{site} \quad \forall i \in \{0, \ldots, n-1 \}\\
    t_j &= \text{puissance de turbinage au temps j} \quad \forall j \in \{0, \ldots, m-1 \}\\
    p_j &= \text{puissance de pompage au temps j} \quad \forall j \in \{0, \ldots, m-1 \}\\
    b_j &= \text{niveau du bassin au temps j} \quad \forall j \in \{0, \ldots, m-1 \}
\end{align*}
$$

Nous avons donc :
$$
\begin{align*}
    \min \quad &\mathrm{cost_{on}} \times \sum_{\mathrm{on}} c_i + \mathrm{cost_{off}} \times \sum_{\mathrm{off}} c_i\\
    &\sum_i c_i \sum_j e_i(j) + \sum_j (\eta \times t_j - p_j) \ge \sum_j \mathrm{consommation_j - a_j}\\
    &\sum_j p_j - t_j = \sum_j a_j\\
    &b_{j+1} = b_j + t_j - p_j + a_j \Leftrightarrow b_{j+1} - b_j - t_j + p_j = a_j\\
    &b_0 = 0.5 \times \mathrm{max\_stockage}\\
    &0 \le p_j \le \mathrm{max\_pompage} \quad \forall j \in \{0, \ldots, m-1 \}\\
    &0 \le t_j \le \mathrm{max\_turbine} \quad \forall j \in \{0, \ldots, m-1 \}\\
    &0 \le c_i \le c_i^\mathrm{max}\\
    &0 \le b_j \le \mathrm{max\_stockage} \quad \forall j \in \{0, \ldots, m-1 \}
\end{align*}
$$

---

In [12]:
# Variable vectors
c = cp.Variable(n)
t = cp.Variable(m)
p = cp.Variable(m)
b = cp.Variable(m)

# Objective function
costs = np.zeros(n)
costs[onshore_idx] = onshore_install_cost
costs[offshore_idx] = offshore_install_cost
f = cp.Minimize(costs.T@c)

constraints = [
    np.sum(rendements_on_off, axis=1)@c + cp.sum(turbine_efficiency*t - p) >= np.sum(np.concatenate(consumptions)) - np.sum(np.concatenate(hydro_productions)),
    cp.sum(p - t) == np.sum(hydro_productions),
    (np.eye(m, k=1) - np.eye(m))@b - t + p == np.sum(hydro_productions, axis=0),
    b[0] == 0.5 * np.sum(max_stock),
    p >= 0,
    p <= np.sum(max_pump_power),
    t >= 0,
    t <= np.sum(max_turbine_power),
    c >= 0,
    c <= max_capacity,
    b >= 0,
    b <= np.sum(max_stock),
]

problem = cp.Problem(f, constraints)

problem.solve()
sol = np.stack((c.value, t.value, p.value, b.value))
np.save("solutions/sol4B.npy", sol)
print(problem.value)

/home/lucas/anaconda3/envs/math-env/lib/python3.12/site-packages/cvxpy/reductions/solvers/solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


inf


In [14]:
sol = np.load("solutions/sol4B.npy", allow_pickle=True)
print(sol[0])

None
